*description*

script to
- load
- filter
- correct
- save
harringtonine data as JSON file for HMM analysis.

Filtering:
- start in the first minute of acquisition;



Add delay between HT and imaging to time directly

In [ ]:
import numpy as np
import pandas as pd
from os.path import join

import json

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import re

import os
import glob

### to be modified by the user ####
input_dir = "path_to_intensity_traces"
output_dir = "path_to_results_directory"
####

from FilterResults import smooth, remove_spikes

from ExtractTraces import get_files_list, get_values, get_unique_IDs

from date_cond2delay import date_cond2delay

#-----------------#


import matplotlib
factor = 0.3
matplotlib.rcParams.update({
        'figure.figsize' : (round(15),round(10)),
        'axes.labelsize' : round(80*factor),
        'axes.titlesize' : round(80*factor),
        'xtick.labelsize' : round(80*factor),
        'ytick.labelsize' : round(80*factor),
        'lines.linewidth' : round(6*factor),
        'lines.markersize' : round(34*factor),
        'legend.fontsize' : round(60*factor),
        'axes.linewidth' : round(10*factor),
        'xtick.major.size': round(20*factor),
        'ytick.major.size': round(20*factor),
        'xtick.major.width': round(8*factor),
        'ytick.major.width': round(8*factor),
        })

In [ ]:
def is_translated(y, thr):
    """
    Parameters
    ----------
    y : 1d array
        green fluorescent track.
    thr : real
        threshold discriminating translation from noise.

    Returns
    -------
    int: how many data points are above threshold.

    """
    ysmoo = smooth(y, w=15, mean=False)
    mask = ysmoo > thr
    return len(ysmoo[mask])


def estimate_n_translated(Y, thr):
    n_translated = 0
    for y in Y:
        if is_translated(y, thr):
            n_translated += 1
    return n_translated

def get_unique_IDs(files):
    
    unique_IDs = []
    
    for f in files:
        
        ID = re.search(r'track(\d+)', f).group(1)
        stage = re.search(r's(\d)', f).group(1)
        
        unique_IDs.append(int(stage+ID))
        
    res = np.array(unique_IDs)
    
    return res

In [ ]:
### modify by the user
sample = "sample_name"
dates = ["date1", "date2"] # list of experiment dates for sample
#####

model_name = "gaussian_plane_v1" #"gaussian_plane"
spec = ""#"_ff"
nframes = 271
dt = 20 # sec

time = np.arange(nframes) * 20 # time is seconds

date2green = {} # dictionary containing date -> array of traces
date2red = {} # dictionary containing date -> array of red traces
date2IDs = {}

date2gw = {} # green spot width
date2rw = {} # red spot width

_, axs = plt.subplots(2, figsize=(7,10), sharex=True)
for i, date in enumerate(dates):

    #-------- Load data ----------#
    print(date)
    files = get_files_list(date, sample, input_dir, model_name, spec, stage='*')
    print(files)

    new_files, green_traces, red_traces, mask = get_values(files, nframes)
    trackIDs = get_unique_IDs(new_files) #
    #print(trackIDs)
    print("# tracks ", len(green_traces))
    
    _, green_w, red_w, _ = get_values(files, nframes, column_name="sigma (um)")
    print("# tracks ", len(green_w))
    
    green_traces = green_traces[mask]
    red_traces = red_traces[mask]
    reen_w = green_w[mask]
    red_w = red_w[mask]
    trackIDs = trackIDs[mask] ## !!! without this the trackIDs are incorrect
    
    #--- delay ----#
    
    try:
        delay = date_cond2delay[(date, sample)]
        print(delay)
    except KeyError:
        print("Key Error: ", date, sample)
        delay = 20

    #----------- Filter traces ----------#

    final_green = []
    final_red = []
    final_IDs = []
    
    final_gw = []
    final_rw = []
    
    for n in range(len(green_traces)):
        
        y = green_traces[n]
        mask = ~np.isnan(y)

        axs[0].plot(time/60., y, c='g', alpha=0.2)
        axs[1].plot(time/60., red_traces[n], c='r', alpha=0.2)
        
        
        if time[mask][0] + delay <= 100: # starting at most 100s after harringtonine # this if you are looking at harringtonine data
                                         # > 0 (no filtering) for steady state data 
        
            final_green.append(y)
            final_red.append(red_traces[n])
            final_IDs.append(trackIDs[n])
            final_gw.append(green_w[n])
            final_rw.append(red_w[n])

    plt.xlim(-1, 20)
    axs[0].set_ylim(-10, 300)
    axs[1].set_ylim(-10, 300)
   
    plt.xlabel("time (min)")
    plt.ylabel("intensity (au)")

    final_green = np.array(final_green)
    final_red = np.array(final_red)
    final_IDs = np.array(final_IDs)
    
    final_gw = np.array(final_gw)
    final_rw = np.array(final_rw)
    print(final_IDs)
    
    ntraces = len(final_green)
    
    print("# filtered tracks ", ntraces, "\n")
    
    #---- Save into dictionary ---- #
    
    date2green[date] = final_green
    date2red[date] = final_red
    date2IDs[date] = final_IDs
    date2gw[date] = final_gw
    date2rw[date] = final_rw

# Run-off starts after 60s from HT addition

Here we save data as JSON file in a format that can be used as input for the Stan HMM code.
Time t = 0 correspond to 60s after harringtonine addition.

The red traces are also added to the JSON file even if not necessary to the HMM analysis (at present).

In [ ]:
for nd, date in enumerate(list(sample2dates[sample])):
    print(date)
    
    Yg = date2green[date]
    Yr = date2red[date]
    IDs = date2IDs[date]
    
    final_green = []
    final_red = []
    final_green_corr = []
    final_ids = []
  
    for n in range(len(Yg)):
            
        yg = Yg[n]
        yr = Yr[n]
        mask = ~np.isnan(yg)
            
        green_corr = remove_spikes(yg)#remove_positive_corr(yg, yr)
        
        trackid = IDs[n]

            
        if np.all(green_corr[mask] >= 0):
                
            final_green_corr.append(green_corr) # corrected
            final_red.append(yr)
            final_green.append(yg)
            final_ids.append(trackid)
            
        
    final_green = np.array(final_green)
    final_red = np.array(final_red)
    final_green_corr = np.array(final_green_corr)
    final_ids = np.array(final_ids, dtype=int)

    ntraces = len(final_green)
    
        
    #----- Remove NaNs and define the time ------#
    
    # Delay from harringtonine addition

    try:
        delay = date_cond2delay[(date, sample)]
        print(delay)
    except KeyError:
        print("Key Error: ", date, sample)
        delay = 20

    times = np.array([time + delay] * ntraces) # TIME AFTER HT ADDITION

    traces_nonan = []
    traces_corr_nonan = []
    times_nonan = []
    reds_nonan = []

    starts = [0]
    for i in range(ntraces):
        
        # 1) we want to remove the intial and final point because they are subject to noise
        # 2) we want also the trace to start not earlier than 60s after HT addition
        
        mask = ~np.isnan(final_green[i])
        time_HT = times[i][mask][1 : -1] # trace time with respect to HT addition
        y = final_green[i][mask][1 : -1]
        ycorr = final_green_corr[i][mask][1 : -1]
        yred = final_red[i][mask][1 : -1]
        
        if time_HT[0] < 60:
            print(time_HT)
            idx = np.arange(len(time_HT))
            i_60 = idx[time_HT >= 60][0]
            time_HT = time_HT[i_60 : ]
            y = y[i_60 : ]
            ycorr = ycorr[i_60 : ]
            yred = yred[i_60 : ]

        traces_nonan.append(y) 
        traces_corr_nonan.append(ycorr)
        
        times_nonan.append(time_HT - 60) # t = 0 correspond to run-off start not HT addition 
    
        reds_nonan.append(yred)
        if i < ntraces-1:
            starts.append(starts[i] + len(traces_nonan[i]))
        
        plt.figure( figsize=(8,5))
        plt.plot((time_HT - 60)/60, y)
        plt.plot((time_HT - 60)/60, yred, c='r')
        plt.title(final_ids[i])
        plt.ylim(0, 200)

    
    
        
    ntraces = len(traces_nonan)
    print(ntraces)
    n_inactive = ntraces - estimate_n_translated(traces_nonan, thr = 10)
    print(" # inactive", n_inactive)
    
    starts = [0]
    for i in range(ntraces-1):
        starts.append(starts[i] + len(traces_nonan[i]))
    
    #----- Concatenate traces ------#
    
    starts = np.array(starts)
    
    traces = np.concatenate(traces_nonan)
    traces_corr = np.concatenate(traces_corr_nonan)
    times = np.concatenate(times_nonan)

    reds = np.concatenate(reds_nonan)

    data = {}

    data = {
        'ndata' : len(traces),
        'ntraces' : ntraces,
        'n_inactive' : n_inactive,
        'I' : traces.tolist(),
        'I_red' : reds.tolist(),
        't' : times.tolist(),
        'starts' : (starts + 1).tolist(),
        'time_step' : dt,
        'delay' : 0,
        'trackIDs' : final_ids.tolist()
        }
    
    #----- Save raw data as JSON file ------#
    
    json_file = join(output_dir, "01HT_raw_{}_{}_ROstart60.json".format(sample, date))
    print("Saving Stan data as json file:", json_file)   
    with open(json_file, "w") as f:
        json.dump(data, f)
        
    # --------------------------------- #
    # corrected traces
    
    data = {
        'ndata' : len(traces_corr),
        'ntraces' : ntraces,
        'n_inactive' : n_inactive,
        'I' : traces_corr.tolist(),
        'I_red' : reds.tolist(),
        't' : times.tolist(),
        'starts' : (starts + 1).tolist(),
        'time_step' : dt,
        'delay' : 0,
        'trackIDs' : final_ids.tolist()
        }
    
    #----- Save corrected data as JSON file ------#
    
    json_file = join(output_dir, "01HT_corr_{}_{}_ROstart60.json".format(sample, date))
    print("Saving Stan data as json file:", json_file)   
    with open(json_file, "w") as f:
        json.dump(data, f)

## Save control and perturbed conditions in the same file

In [ ]:
# Save control and perturbed conditions in the same file

reporter = "COL"
sample="COL-H-AG*_01"
for date in list(sample2dates[sample]):
    
    print(date)
    
    files = []
    files.append("01HT_corr_{}_{}_ROstart60.json".format(reporter+"-H-AG*_01", date))
    files.append("01HT_corr_{}_{}_ROstart60.json".format(reporter+"-H-GC7*_01", date))
    
    # store in a list data per condition
    ndata_l = []
    ntraces_l = []
    finactive_l = []
    I_l = []
    t_l = []
    starts_l = []
    trackIDs_l = []
    
    try:
        for i, file in enumerate(files):

            with open(join(output_dir, file)) as f:
                data = json.load(f)

            ndata_l.append(data["ndata"])
            ntraces_l.append(data["ntraces"])
            finactive_l.append(data["n_inactive"] / data["ntraces"])
            I_l.append(data["I"])
            t_l.append(data["t"])
            trackIDs_l.append(data["trackIDs"])
            
            if i > 0:
                starts = [ndata_l[i - 1] + s for s in data["starts"]]
                starts_l.append(starts)
            else:
                starts_l.append(data["starts"])
                
    except FileNotFoundError as e:
        print(e)
        continue
        
    
    ndata = sum(ndata_l)
    ntraces = sum(ntraces_l)
    f_inactive = finactive_l
    I = np.concatenate(I_l)
    t = np.concatenate(t_l)
    starts = np.concatenate(starts_l)
    trackIDs = np.concatenate(trackIDs_l)
    
    cond_starts = [starts_l[0][0], starts_l[1][0]]
    
    data = {
            'ncond' : 2,
            'ndata' : ndata,
            'ntraces' : ntraces,
            'f_inactive' : f_inactive,
            'I' : I.tolist(),
            't' : t.tolist(),
            'starts' : starts.tolist(),
            'cond_starts' : cond_starts,
            'time_step' : dt,
            'trackIDs' : trackIDs.tolist()
            }
    
    #----- Save data as JSON file ------#
    
    json_file = join(output_dir, "01HT_corr_{}_{}_ROstart60.json".format(reporter, date))
    print("Saving Stan data as json file:", json_file)   
    with open(json_file, "w") as f:
        json.dump(data, f)